In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import nibabel as nib
import numpy as np

!pip install SimpleITK   # not installed by default
import SimpleITK as sitk

from scipy.ndimage.interpolation import rotate

In [ ]:
def compute_nii_label_volume(nii_filepath):
  nii_img = nib.load(nii_filepath)
  print('affine:', nii_img.affine)
  voxel_spacing = [nii_img.affine[i][i] for i in range(3)]
  voxel_volume = np.prod(voxel_spacing)
  print('voxel_spacing:', voxel_spacing)
  print('voxel_volume:', voxel_volume, 'mm3')
  label_data = np.array(np.rint(nii_img.get_fdata()), dtype=np.int8)
  
  label_values = np.unique(label_data)
  print(label_values)  # this is used to compute the volume of all labels [0,1,2 etc.] that are present
  volumes = [0 for n in label_values]
  for i,k in enumerate(label_values):
    volumes[i] = voxel_volume * np.sum(np.where(label_data == k, 1, 0))

  return volumes

In [ ]:
imgpath = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/STOPIT/images/'
segpath = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/STOPIT/segmentations/'
print(os.path.exists(imgpath))
print(os.path.exists(segpath))

True
True


In [ ]:
imgfiles = [name for name in os.listdir(imgpath) if '.nii.gz' in name]
segfiles = [name for name in os.listdir(segpath) if '.nii.gz' in name]

print(len(imgfiles))
print(len(segfiles))

segfiles = sorted(imgfiles, key=lambda x: imgfiles.index(x))

for i, name in enumerate(segfiles):
  segfiles[i] = "STOPIT-" + name.replace("-Baseline", "")

for i, j in zip(imgfiles, segfiles):
  print(i + "  " + j)

count1 = 0
count2 = 0
angle = 0

outpath_cw_img = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/STOPIT/cw/images/'
outpath_cw_seg = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/STOPIT/cw/segmentations/'
outpath_ccw_img = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/STOPIT/ccw/images/'
outpath_ccw_seg = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/STOPIT/ccw/segmentations/'

os.makedirs(outpath_cw_img)
os.makedirs(outpath_cw_seg)
os.makedirs(outpath_ccw_img)
os.makedirs(outpath_ccw_seg)

45
45
01-01-002-Baseline.nii.gz  STOPIT-01-01-002.nii.gz
01-01-003-Baseline.nii.gz  STOPIT-01-01-003.nii.gz
01-01-004-Baseline.nii.gz  STOPIT-01-01-004.nii.gz
01-01-007-Baseline.nii.gz  STOPIT-01-01-007.nii.gz
01-01-009-Baseline.nii.gz  STOPIT-01-01-009.nii.gz
01-01-015-Baseline.nii.gz  STOPIT-01-01-015.nii.gz
01-01-018-Baseline.nii.gz  STOPIT-01-01-018.nii.gz
01-01-019-Baseline.nii.gz  STOPIT-01-01-019.nii.gz
03-01-001-Baseline.nii.gz  STOPIT-03-01-001.nii.gz
03-01-003-Baseline.nii.gz  STOPIT-03-01-003.nii.gz
03-01-004-Baseline.nii.gz  STOPIT-03-01-004.nii.gz
03-01-007-Baseline.nii.gz  STOPIT-03-01-007.nii.gz
03-01-008-Baseline.nii.gz  STOPIT-03-01-008.nii.gz
04-01-002-Baseline.nii.gz  STOPIT-04-01-002.nii.gz
04-01-008-Baseline.nii.gz  STOPIT-04-01-008.nii.gz
04-01-009-Baseline.nii.gz  STOPIT-04-01-009.nii.gz
05-01-002-Baseline.nii.gz  STOPIT-05-01-002.nii.gz
05-01-004-Baseline.nii.gz  STOPIT-05-01-004.nii.gz
06-01-001-Baseline.nii.gz  STOPIT-06-01-001.nii.gz
08-01-010-Baseline.nii.gz

FileExistsError: ignored

In [ ]:
#clockwise deg image/segmentation
for i, j in zip(imgfiles, segfiles):
  count1 = count1 + 1
  angle=np.random.randint(1, 11)
  
  #image
  file1 = os.path.join(imgpath, i)
  img = nib.load(file1) 
  img_data = img.get_fdata()
  rotated_cw = rotate(img_data, angle, reshape=True)
  #affine_cw = rotate(img.affine, angle=10)
  print(i)
  print(img.affine)
  output_cw = nib.Nifti1Image(rotated_cw, img.affine, img.header)
  nib.save(output_cw, os.path.join(outpath_cw_img, i))
  print("Saved!")
  
  #seg
  count2 = count2 + 1
  file2 = os.path.join(segpath, j)
  seg = nib.load(file2)
  seg_data = seg.get_fdata()
  rotated_cw = rotate(seg_data, angle)
  #affine_cw = rotate(seg.affine, angle=10)
  print(j)
  print(seg.affine)
  output_cw = nib.Nifti1Image(rotated_cw, seg.affine, seg.header)
  nib.save(output_cw, os.path.join(outpath_cw_seg, j))
  print("Saved!")

print("Img files: " + str(count1))
print("Seg files: " + str(count2))

01-01-002-Baseline.nii.gz
[[-4.68749957e-01 -1.97063797e-04 -3.78650014e-04  1.19765625e+02]
 [ 1.97063797e-04 -4.40480873e-01 -1.74704988e+00  2.99218872e+02]
 [ 3.47476652e-05 -1.60321939e-01  4.79998019e+00 -5.82739746e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Saved!
STOPIT-01-01-002.nii.gz
[[-4.68749957e-01 -1.97063797e-04 -3.78650014e-04  1.19765625e+02]
 [ 1.97063797e-04 -4.40480873e-01 -1.74704988e+00  2.99218872e+02]
 [ 3.47476652e-05 -1.60321939e-01  4.79998019e+00 -5.82739746e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Saved!


KeyboardInterrupt: ignored

In [ ]:
#counter clockwise deg image
count1 = 0
count2 = 0

for i, j in zip(imgfiles, segfiles):
  count1 = count1 + 1
  angle = np.random.randint(-10, 0)

  #img
  file1 = os.path.join(imgpath, i)
  img = nib.load(file1)
  img_data = img.get_fdata()
  rotated_ccw = rotate(img_data, angle)
  #affine_cw = rotate(img.affine, angle=10)
  print(i)
  print(img.affine)
  output_ccw = nib.Nifti1Image(rotated_ccw, img.affine, img.header)
  nib.save(output_ccw, os.path.join(outpath_ccw_img, i))
  print("Saved!")

  #seg
  count2 = count2 + 1
  file2 = os.path.join(segpath, j)
  seg = nib.load(file2)
  seg_data = seg.get_fdata()
  rotated_ccw = rotate(seg_data, angle)
  #affine_cw = rotate(seg.affine, angle=10)
  print(j)
  print(seg.affine)
  output_ccw = nib.Nifti1Image(rotated_ccw, seg.affine, seg.header)
  nib.save(output_ccw, os.path.join(outpath_ccw_seg, j))
  print("Saved!")

print(count1)
print(count2)

01-01-019-Baseline.nii.gz
[[ -0.42968801   0.           0.         110.        ]
 [  0.          -0.40748327  -1.672831   104.31600189]
 [  0.          -0.13634209   4.99956135   2.90400004]
 [  0.           0.           0.           1.        ]]
Saved!
STOPIT-01-01-019.nii.gz
[[ -0.42968801   0.           0.         110.        ]
 [  0.          -0.40748327  -1.672831   104.31600189]
 [  0.          -0.13634209   4.99956135   2.90400004]
 [  0.           0.           0.           1.        ]]
Saved!
04-01-006-Baseline.nii.gz
[[ -0.48828101   0.           0.         125.        ]
 [  0.          -0.4460668   -2.22607236 119.89299774]
 [  0.          -0.198602     4.99983366  15.63000011]
 [  0.           0.           0.           1.        ]]
Saved!
STOPIT-04-01-006.nii.gz
[[ -0.48828101   0.           0.         125.        ]
 [  0.          -0.44606681  -2.22607221 119.89299774]
 [  0.          -0.19860199   4.99983373  15.63000011]
 [  0.           0.           0.           1.      

In [ ]:
path = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/SPOTLIGHT/ccw/images'
files = os.listdir(path)
print(len(files))

50


In [ ]:
path = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/PREDICT/segmentations/'
path_cw = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/PREDICT/cw/segmentations/'
path_ccw = '/content/drive/My Drive/Hematoma Expansion Cohorts/Final Images/dataset/PREDICT/ccw/segmentations/'
files = [name for name in os.listdir(path) if '.nii.gz' in name]
files_cw = [name for name in os.listdir(path_cw) if '.nii.gz' in name]
files_ccw = [name for name in os.listdir(path_ccw) if '.nii.gz' in name]

files_cw = sorted(files, key=lambda x: files.index(x))
for i, file in enumerate(files_cw):
  files_cw[i] = file.split(".")[0] + "_cw.nii.gz"
  print(files_cw[i])

for i,j in zip(files[0:5],files_cw[0:5]): 
  print("-------Original-------: " + i)
  compute_nii_label_volume(os.path.join(path, i))
  print("-------Rotated-------: " + j)
  compute_nii_label_volume(os.path.join(path_cw, j))

PREDICT-01-005_cw.nii.gz
PREDICT-01-029_cw.nii.gz
PREDICT-01-049_cw.nii.gz
PREDICT-01-092_cw.nii.gz
PREDICT-01-101_cw.nii.gz
PREDICT-04-026_cw.nii.gz
PREDICT-07-003_cw.nii.gz
PREDICT-11-010_cw.nii.gz
PREDICT-01-025_cw.nii.gz
PREDICT-01-033_cw.nii.gz
PREDICT-01-063_cw.nii.gz
PREDICT-01-069_cw.nii.gz
PREDICT-09-007_cw.nii.gz
PREDICT-13-004_cw.nii.gz
PREDICT-01-010_cw.nii.gz
PREDICT-01-040_cw.nii.gz
PREDICT-01-048_cw.nii.gz
PREDICT-01-059_cw.nii.gz
PREDICT-01-066_cw.nii.gz
PREDICT-04-025_cw.nii.gz
PREDICT-09-017_cw.nii.gz
PREDICT-13-005_cw.nii.gz
PREDICT-01-011_cw.nii.gz
PREDICT-01-035_cw.nii.gz
PREDICT-01-096_cw.nii.gz
PREDICT-09-012_cw.nii.gz
PREDICT-01-028_cw.nii.gz
PREDICT-01-072_cw.nii.gz
PREDICT-11-006_cw.nii.gz
PREDICT-04-004_cw.nii.gz
PREDICT-04-002_cw.nii.gz
PREDICT-10-010_cw.nii.gz
PREDICT-05-025_cw.nii.gz
PREDICT-05-038_cw.nii.gz
PREDICT-05-039_cw.nii.gz
PREDICT-01-050_cw.nii.gz
PREDICT-01-088_cw.nii.gz
PREDICT-04-027_cw.nii.gz
PREDICT-05-004_cw.nii.gz
PREDICT-05-005_cw.nii.gz
